In [10]:
def create_flexible_patent_regression_model(input_shape=(224, 224, 3)):
    """
    Create flexible patent-based regression model that can handle:
    - Top view only
    - Side view only  
    - Both views (dual-view)
    """
    
    # Single image input (can be top, side, or both)
    image_input = Input(shape=input_shape, name='image_input')
    
    # CNN backbone for feature extraction
    base_cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_cnn.trainable = False
    
    # Extract features
    features = base_cnn(image_input)
    features = GlobalAveragePooling2D(name='gap')(features)
    
    # Feature processing
    processed = Dense(512, activation='relu', name='dense1')(features)
    processed = BatchNormalization(name='bn1')(processed)
    processed = Dropout(0.3, name='dropout1')(processed)
    
    processed = Dense(256, activation='relu', name='dense2')(processed)
    processed = BatchNormalization(name='bn2')(processed)
    processed = Dropout(0.3, name='dropout2')(processed)
    
    # Final regression output
    parameter_predictions = Dense(4, activation='linear', name='parameter_regression')(processed)
    
    # Create model
    model = Model(inputs=image_input, outputs=parameter_predictions, 
                  name='FlexiblePatentRegressionModel')
    
    return model

# Alternative: Update the predictor class to handle flexible inputs
class FlexiblePatentRegressionPredictor:
    """
    Flexible predictor that can work with top-only, side-only, or dual-view images
    """
    
    def __init__(self, model_path, data_path='./Materials_data'):
        self.model_path = model_path
        self.data_path = data_path
        self.model = None
        self.reference_data = None
        self.param_scaler = None
        
        # Parameter names and weights from patent
        self.param_names = ['弯曲强度', '强度', '形变强度', '形变率']
        self.param_names_en = ['Bending_Strength', 'Strength', 'Deformation_Strength', 'Deformation_Rate']
        self.param_weights = np.array([1.0, 0.6, 0.6, 0.3])
        
    def predict_with_single_image(self, image_path, view_type='auto'):
        """
        Predict parameters using a single image
        
        Args:
            image_path: Path to the image
            view_type: 'top', 'side', or 'auto' (auto-detect from filename)
        """
        
        if self.model is None:
            self.load_model_and_data()
        
        # Auto-detect view type from filename if not specified
        if view_type == 'auto':
            if '-1.' in image_path or 'top' in image_path.lower():
                view_type = 'top'
            elif '-2.' in image_path or 'side' in image_path.lower():
                view_type = 'side'
            else:
                view_type = 'unknown'
        
        # Preprocess image
        img = self.preprocess_single_image(image_path)
        if img is None:
            return None
        
        # Make prediction - use the same image for both inputs in dual-view model
        img_input = np.expand_dims(img, axis=0)
        
        try:
            # If using the original dual-view model, provide same image for both inputs
            predictions_normalized = self.model.predict([img_input, img_input], verbose=0)
            
            # Denormalize predictions
            if self.param_scaler is not None:
                predictions_original = self.param_scaler.inverse_transform(predictions_normalized)
            else:
                predictions_original = predictions_normalized
            
            return {
                'predicted_parameters_original': predictions_original[0],
                'predicted_parameters_normalized': predictions_normalized[0],
                'parameter_names': self.param_names,
                'parameter_names_en': self.param_names_en,
                'image_path': image_path,
                'view_type': view_type,
                'note': f'Prediction based on {view_type} view image'
            }
            
        except Exception as e:
            print(f"Error during prediction: {e}")
            return None
    
    def preprocess_single_image(self, image_path):
        """Preprocess a single image"""
        try:
            if not os.path.exists(image_path):
                return None
            
            img = load_img(image_path, target_size=(224, 224), color_mode='rgb')
            img_array = img_to_array(img) / 255.0
            return img_array
            
        except Exception as e:
            print(f"Error preprocessing image {image_path}: {e}")
            return None
    
    def load_model_and_data(self):
        """Load the trained model and reference data"""
        print("Loading model and reference data...")
        
        try:
            # Load model with custom objects
            parameter_weights = self.param_weights
            custom_objects = {
                'weighted_mse': parameter_weighted_mse_loss(parameter_weights),
                'weighted_mae': parameter_weighted_mae_loss(parameter_weights)
            }
            
            self.model = tf.keras.models.load_model(self.model_path, custom_objects=custom_objects)
            print(f"Model loaded from {self.model_path}")
            
            # Load reference data for similarity comparison
            data_loader = PatentRegressionDataLoader(self.data_path, normalize_params=True)
            top_images, side_images, parameter_values, folder_names, image_paths = data_loader.load_regression_data()
            
            self.param_scaler = data_loader.param_scaler
            
            self.reference_data = {
                'parameters_normalized': parameter_values,
                'folder_names': folder_names,
                'image_paths': image_paths
            }
            
            if self.param_scaler is not None:
                self.reference_data['parameters_original'] = self.param_scaler.inverse_transform(parameter_values)
            else:
                self.reference_data['parameters_original'] = parameter_values
            
            print(f"Reference dataset loaded: {len(folder_names)} samples")
            
        except Exception as e:
            print(f"Error loading model or data: {e}")
            raise

# Patent-Based Material Parameter Regression
Complete implementation for continuous parameter prediction from dual-view images

In [11]:
# Imports
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Concatenate, Dropout, 
                                   GlobalAveragePooling2D, BatchNormalization,
                                   Lambda, Add, Multiply)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import matplotlib.pyplot as plt
import imghdr
from collections import Counter

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Data Loader Class

In [12]:
class PatentRegressionDataLoader:
    """
    Regression data loader following patent specifications:
    - Predicts continuous parameter values from dual-view images
    - Target: [弯曲强度, 强度, 形变强度, 形变率] (continuous values 0-100)
    - Patent methodology: Replace physical testing with AI image analysis
    """
    
    def __init__(self, data_path, image_size=(224, 224), normalize_params=True):
        self.data_path = data_path
        self.image_size = image_size
        self.normalize_params = normalize_params
        
        # Parameter importance weights from patent findings
        # 弯曲强度(最显著), 强度(较显著), 形变强度(较显著), 形变率(不显著)
        self.parameter_weights = np.array([1.0, 0.6, 0.6, 0.3], dtype=np.float32)
        
        # Parameter names for reference
        self.param_names = ['弯曲强度', '强度', '形变强度', '形变率']
        self.param_names_en = ['Bending_Strength', 'Strength', 'Deformation_Strength', 'Deformation_Rate']
        
        # Parameter scalers for normalization
        self.param_scaler = None
        
    def extract_parameters_from_folder(self, folder_name):
        """
        Extract continuous physical parameters from folder name for regression
        Format: "1-20.20.20.20-1" -> [20.0, 20.0, 20.0, 20.0]
        Returns raw parameter values (not discretized)
        """
        try:
            parts = folder_name.split('-')
            if len(parts) >= 2:
                param_str = parts[1]  # "20.20.20.20"
                params = [float(x) for x in param_str.split('.')]
                if len(params) >= 4:
                    # Validate parameter ranges and return raw values
                    validated_params = []
                    for i, param in enumerate(params[:4]):
                        # Ensure parameters are in valid range [0, 100]
                        validated_params.append(max(0.0, min(100.0, param)))
                    return validated_params
        except Exception as e:
            print(f"Error extracting parameters from {folder_name}: {e}")
        
        # Default parameters if extraction fails
        return [0.0, 20.0, 20.0, 20.0]
    
    def load_and_preprocess_image(self, image_path):
        """Load and preprocess image according to patent specifications"""
        try:
            if not os.path.exists(image_path):
                return None
                
            # Check if valid image
            if imghdr.what(image_path) not in ['jpeg', 'jpg', 'png']:
                return None
                
            # Load image as RGB (patent specifies color capture)
            img = load_img(image_path, target_size=self.image_size, color_mode='rgb')
            img_array = img_to_array(img) / 255.0  # Normalize to [0,1]
            return img_array
            
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None
    
    def load_regression_data(self):
        """
        Load dual-view images with continuous parameter targets for regression
        Returns: top_images, side_images, parameter_values, folder_names
        """
        top_images = []
        side_images = []
        parameter_values = []
        folder_names = []
        image_paths = []
        
        print("Loading patent-based regression data...")
        print("Target: Continuous parameter prediction [弯曲强度, 强度, 形变强度, 形变率]")
        
        for folder in sorted(os.listdir(self.data_path)):
            folder_path = os.path.join(self.data_path, folder)
            
            if not os.path.isdir(folder_path):
                continue
                
            # Extract continuous physical parameters from folder name
            params = self.extract_parameters_from_folder(folder)
            
            # Find image pairs (top-view and side-view)
            files = os.listdir(folder_path)
            image_pairs = {}
            
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    if '-1.' in file:  # Top view
                        base_name = file.replace('-1.', '.')
                        if base_name not in image_pairs:
                            image_pairs[base_name] = {}
                        image_pairs[base_name]['top'] = file
                    elif '-2.' in file:  # Side view
                        base_name = file.replace('-2.', '.')
                        if base_name not in image_pairs:
                            image_pairs[base_name] = {}
                        image_pairs[base_name]['side'] = file
            
            # Load valid pairs for regression training
            for base_name, pair in image_pairs.items():
                if 'top' in pair and 'side' in pair:
                    top_path = os.path.join(folder_path, pair['top'])
                    side_path = os.path.join(folder_path, pair['side'])
                    
                    top_img = self.load_and_preprocess_image(top_path)
                    side_img = self.load_and_preprocess_image(side_path)
                    
                    if top_img is not None and side_img is not None:
                        top_images.append(top_img)
                        side_images.append(side_img)
                        parameter_values.append(params)
                        folder_names.append(folder)
                        image_paths.append((top_path, side_path))
        
        # Convert to numpy arrays
        top_images = np.array(top_images)
        side_images = np.array(side_images)
        parameter_values = np.array(parameter_values, dtype=np.float32)
        
        print(f"Loaded {len(top_images)} image pairs for regression")
        print(f"Parameter shape: {parameter_values.shape}")
        
        # Store data statistics for analysis
        self._store_regression_statistics(parameter_values, folder_names)
        
        # Normalize parameters if requested
        if self.normalize_params:
            parameter_values = self._normalize_parameters(parameter_values)
            print("Parameters normalized to [0,1] range")
        
        return top_images, side_images, parameter_values, folder_names, image_paths
    
    def _normalize_parameters(self, parameter_values):
        """Normalize parameters to [0,1] range for better training stability"""
        if self.param_scaler is None:
            # Use MinMaxScaler to normalize to [0,1] range
            self.param_scaler = MinMaxScaler()
            normalized_params = self.param_scaler.fit_transform(parameter_values)
        else:
            normalized_params = self.param_scaler.transform(parameter_values)
        
        return normalized_params.astype(np.float32)
    
    def denormalize_parameters(self, normalized_params):
        """Convert normalized parameters back to original scale"""
        if self.param_scaler is None:
            print("Warning: No scaler available for denormalization")
            return normalized_params
        
        return self.param_scaler.inverse_transform(normalized_params)
    
    def _store_regression_statistics(self, parameter_values, folder_names):
        """Store regression data statistics for analysis"""
        self._data_stats = {
            'total_samples': len(parameter_values),
            'unique_folders': len(set(folder_names)),
            'parameter_statistics': {}
        }
        
        print("\nRegression Target Statistics:")
        for i, (name_zh, name_en) in enumerate(zip(self.param_names, self.param_names_en)):
            param_vals = parameter_values[:, i]
            stats = {
                'mean': np.mean(param_vals),
                'std': np.std(param_vals),
                'min': np.min(param_vals),
                'max': np.max(param_vals),
                'median': np.median(param_vals)
            }
            self._data_stats['parameter_statistics'][name_en] = stats
            
            print(f"  {name_zh} ({name_en}):")
            print(f"    Range: [{stats['min']:.1f}, {stats['max']:.1f}]")
            print(f"    Mean±Std: {stats['mean']:.1f}±{stats['std']:.1f}")
        
        # Check parameter distribution
        print(f"\nParameter Distribution Analysis:")
        for i, name in enumerate(self.param_names_en):
            unique_vals = len(np.unique(parameter_values[:, i]))
            print(f"  {name}: {unique_vals} unique values")

## Model Architecture

In [13]:
def create_patent_regression_model(input_shape=(224, 224, 3)):
    """
    Create patent-based regression model for continuous parameter prediction
    
    Architecture:
    - Dual-view CNN feature extraction (ResNet50 backbone)
    - Multi-modal feature fusion
    - Regression head for 4 continuous parameters
    """
    
    # Dual-view image inputs
    top_view = Input(shape=input_shape, name='top_view')
    side_view = Input(shape=input_shape, name='side_view')
    
    # Shared CNN backbone for feature extraction
    base_cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_cnn.trainable = False  # Start with frozen weights for transfer learning
    
    # Extract features from both views
    top_features = base_cnn(top_view)
    side_features = base_cnn(side_view)
    
    # Global Average Pooling
    top_features = GlobalAveragePooling2D(name='top_gap')(top_features)
    side_features = GlobalAveragePooling2D(name='side_gap')(side_features)
    
    # View-specific processing
    top_processed = Dense(512, activation='relu', name='top_dense')(top_features)
    top_processed = BatchNormalization(name='top_bn')(top_processed)
    top_processed = Dropout(0.3, name='top_dropout')(top_processed)
    
    side_processed = Dense(512, activation='relu', name='side_dense')(side_features)
    side_processed = BatchNormalization(name='side_bn')(side_processed)
    side_processed = Dropout(0.3, name='side_dropout')(side_processed)
    
    # Multi-view fusion with attention
    combined_features = Concatenate(name='view_fusion')([top_processed, side_processed])
    
    # Attention mechanism for view importance
    attention_weights = Dense(1024, activation='softmax', name='view_attention')(combined_features)
    attended_features = Multiply(name='attended_features')([combined_features, attention_weights])
    
    # Regression head with parameter-specific branches
    fusion_dense = Dense(512, activation='relu', name='fusion_dense1')(attended_features)
    fusion_dense = BatchNormalization(name='fusion_bn1')(fusion_dense)
    fusion_dense = Dropout(0.4, name='fusion_dropout1')(fusion_dense)
    
    fusion_dense = Dense(256, activation='relu', name='fusion_dense2')(fusion_dense)
    fusion_dense = BatchNormalization(name='fusion_bn2')(fusion_dense)
    fusion_dense = Dropout(0.3, name='fusion_dropout2')(fusion_dense)
    
    # Final regression output - 4 continuous parameters
    # Using linear activation for regression (no bounds imposed here)
    parameter_predictions = Dense(4, activation='linear', name='parameter_regression')(fusion_dense)
    
    # Create model
    model = Model(inputs=[top_view, side_view], outputs=parameter_predictions, 
                  name='PatentRegressionModel')
    
    return model

## Loss Functions

In [14]:
def parameter_weighted_mse_loss(parameter_weights):
    """
    Create parameter-weighted MSE loss based on patent importance findings
    
    Args:
        parameter_weights: Array of importance weights [弯曲强度, 强度, 形变强度, 形变率]
    """
    def weighted_mse(y_true, y_pred):
        # Compute squared errors for each parameter
        squared_errors = tf.square(y_true - y_pred)
        
        # Apply importance weights from patent
        weighted_errors = squared_errors * tf.constant(parameter_weights, dtype=tf.float32)
        
        # Return mean weighted error
        return tf.reduce_mean(weighted_errors)
    
    return weighted_mse

def parameter_weighted_mae_loss(parameter_weights):
    """
    Create parameter-weighted MAE loss for more robust training
    """
    def weighted_mae(y_true, y_pred):
        # Compute absolute errors for each parameter
        abs_errors = tf.abs(y_true - y_pred)
        
        # Apply importance weights from patent
        weighted_errors = abs_errors * tf.constant(parameter_weights, dtype=tf.float32)
        
        # Return mean weighted error
        return tf.reduce_mean(weighted_errors)
    
    return weighted_mae

## Training Functions

In [15]:
def create_regression_callbacks():
    """Create callbacks for regression training"""
    return [
        EarlyStopping(
            monitor='val_loss',
            patience=25,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            'best_patent_regression_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

def evaluate_regression_model(model, X_test, y_test, param_scaler=None):
    """Evaluate regression model with comprehensive metrics"""
    
    # Make predictions
    y_pred = model.predict(X_test, verbose=0)
    
    # Denormalize if scaler is provided
    if param_scaler is not None:
        y_test_orig = param_scaler.inverse_transform(y_test)
        y_pred_orig = param_scaler.inverse_transform(y_pred)
    else:
        y_test_orig = y_test
        y_pred_orig = y_pred
    
    print("\nRegression Evaluation Results:")
    print("="*60)
    
    # Parameter names
    param_names = ['Bending_Strength', 'Strength', 'Deformation_Strength', 'Deformation_Rate']
    param_names_zh = ['弯曲强度', '强度', '形变强度', '形变率']
    
    # Overall metrics
    overall_mse = mean_squared_error(y_test, y_pred)
    overall_mae = mean_absolute_error(y_test, y_pred)
    overall_r2 = r2_score(y_test, y_pred)
    
    print(f"Overall Metrics (Normalized):")
    print(f"  MSE: {overall_mse:.4f}")
    print(f"  MAE: {overall_mae:.4f}")
    print(f"  R²: {overall_r2:.4f}")
    
    # Per-parameter metrics
    print(f"\nPer-Parameter Metrics (Original Scale):")
    for i, (name_en, name_zh) in enumerate(zip(param_names, param_names_zh)):
        mse = mean_squared_error(y_test_orig[:, i], y_pred_orig[:, i])
        mae = mean_absolute_error(y_test_orig[:, i], y_pred_orig[:, i])
        r2 = r2_score(y_test_orig[:, i], y_pred_orig[:, i])
        
        print(f"  {name_zh} ({name_en}):")
        print(f"    MSE: {mse:.4f}")
        print(f"    MAE: {mae:.4f}")
        print(f"    R²: {r2:.4f}")
    
    return {
        'overall_mse': overall_mse,
        'overall_mae': overall_mae,
        'overall_r2': overall_r2,
        'predictions': y_pred,
        'predictions_original': y_pred_orig,
        'true_values': y_test,
        'true_values_original': y_test_orig
    }

def plot_regression_results(results, save_path='regression_results.png'):
    """Plot regression prediction results"""
    
    y_true_orig = results['true_values_original']
    y_pred_orig = results['predictions_original']
    
    param_names = ['Bending Strength', 'Strength', 'Deformation Strength', 'Deformation Rate']
    param_names_zh = ['弯曲强度', '强度', '形变强度', '形变率']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    for i, (ax, name_en, name_zh) in enumerate(zip(axes, param_names, param_names_zh)):
        # Scatter plot of true vs predicted
        ax.scatter(y_true_orig[:, i], y_pred_orig[:, i], alpha=0.6, color=f'C{i}')
        
        # Perfect prediction line
        min_val = min(y_true_orig[:, i].min(), y_pred_orig[:, i].min())
        max_val = max(y_true_orig[:, i].max(), y_pred_orig[:, i].max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')
        
        # Calculate R²
        r2 = r2_score(y_true_orig[:, i], y_pred_orig[:, i])
        mae = mean_absolute_error(y_true_orig[:, i], y_pred_orig[:, i])
        
        ax.set_xlabel(f'True {name_en}')
        ax.set_ylabel(f'Predicted {name_en}')
        ax.set_title(f'{name_zh}\nR² = {r2:.3f}, MAE = {mae:.2f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Regression results plotted and saved to {save_path}")

def plot_training_history(history, fine_tune_history=None):
    """Plot training history for regression model"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot loss
    ax1.plot(history.history['loss'], label='Training Loss', color='blue')
    ax1.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    
    if fine_tune_history:
        epochs_offset = len(history.history['loss'])
        ax1.plot(range(epochs_offset, epochs_offset + len(fine_tune_history.history['loss'])),
                fine_tune_history.history['loss'], label='Fine-tune Training', linestyle='--', color='blue')
        ax1.plot(range(epochs_offset, epochs_offset + len(fine_tune_history.history['val_loss'])),
                fine_tune_history.history['val_loss'], label='Fine-tune Validation', linestyle='--', color='orange')
    
    ax1.set_title('Model Loss (Parameter-Weighted MSE)')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot MAE
    ax2.plot(history.history['mae'], label='Training MAE', color='green')
    ax2.plot(history.history['val_mae'], label='Validation MAE', color='red')
    
    if fine_tune_history:
        epochs_offset = len(history.history['mae'])
        ax2.plot(range(epochs_offset, epochs_offset + len(fine_tune_history.history['mae'])),
                fine_tune_history.history['mae'], label='Fine-tune Training MAE', linestyle='--', color='green')
        ax2.plot(range(epochs_offset, epochs_offset + len(fine_tune_history.history['val_mae'])),
                fine_tune_history.history['val_mae'], label='Fine-tune Validation MAE', linestyle='--', color='red')
    
    ax2.set_title('Model MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Mean Absolute Error')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('regression_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

def create_flexible_patent_regression_model(input_shape=(224, 224, 3)):
    """
    Create flexible patent-based regression model that can handle:
    - Top view only
    - Side view only  
    - Both views (dual-view)
    """
    
    # Single image input (can be top, side, or both)
    image_input = Input(shape=input_shape, name='image_input')
    
    # CNN backbone for feature extraction
    base_cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_cnn.trainable = False
    
    # Extract features
    features = base_cnn(image_input)
    features = GlobalAveragePooling2D(name='gap')(features)
    
    # Feature processing
    processed = Dense(512, activation='relu', name='dense1')(features)
    processed = BatchNormalization(name='bn1')(processed)
    processed = Dropout(0.3, name='dropout1')(processed)
    
    processed = Dense(256, activation='relu', name='dense2')(processed)
    processed = BatchNormalization(name='bn2')(processed)
    processed = Dropout(0.3, name='dropout2')(processed)
    
    # Final regression output
    parameter_predictions = Dense(4, activation='linear', name='parameter_regression')(processed)
    
    # Create model
    model = Model(inputs=image_input, outputs=parameter_predictions, 
                  name='FlexiblePatentRegressionModel')
    
    return model

# Alternative: Update the predictor class to handle flexible inputs
class FlexiblePatentRegressionPredictor:
    """
    Flexible predictor that can work with top-only, side-only, or dual-view images
    """
    
    def __init__(self, model_path, data_path='./Materials_data'):
        self.model_path = model_path
        self.data_path = data_path
        self.model = None
        self.reference_data = None
        self.param_scaler = None
        
        # Parameter names and weights from patent
        self.param_names = ['弯曲强度', '强度', '形变强度', '形变率']
        self.param_names_en = ['Bending_Strength', 'Strength', 'Deformation_Strength', 'Deformation_Rate']
        self.param_weights = np.array([1.0, 0.6, 0.6, 0.3])
        
    def predict_with_single_image(self, image_path, view_type='auto'):
        """
        Predict parameters using a single image
        
        Args:
            image_path: Path to the image
            view_type: 'top', 'side', or 'auto' (auto-detect from filename)
        """
        
        if self.model is None:
            self.load_model_and_data()
        
        # Auto-detect view type from filename if not specified
        if view_type == 'auto':
            if '-1.' in image_path or 'top' in image_path.lower():
                view_type = 'top'
            elif '-2.' in image_path or 'side' in image_path.lower():
                view_type = 'side'
            else:
                view_type = 'unknown'
        
        # Preprocess image
        img = self.preprocess_single_image(image_path)
        if img is None:
            return None
        
        # Make prediction - use the same image for both inputs in dual-view model
        img_input = np.expand_dims(img, axis=0)
        
        try:
            # If using the original dual-view model, provide same image for both inputs
            predictions_normalized = self.model.predict([img_input, img_input], verbose=0)
            
            # Denormalize predictions
            if self.param_scaler is not None:
                predictions_original = self.param_scaler.inverse_transform(predictions_normalized)
            else:
                predictions_original = predictions_normalized
            
            return {
                'predicted_parameters_original': predictions_original[0],
                'predicted_parameters_normalized': predictions_normalized[0],
                'parameter_names': self.param_names,
                'parameter_names_en': self.param_names_en,
                'image_path': image_path,
                'view_type': view_type,
                'note': f'Prediction based on {view_type} view image'
            }
            
        except Exception as e:
            print(f"Error during prediction: {e}")
            return None
    
    def predict_material_parameters(self, top_image_path, side_image_path):
        """
        Predict parameters using dual-view images (compatible with original predictor)
        """
        if self.model is None:
            self.load_model_and_data()
        
        # Preprocess both images
        top_img = self.preprocess_single_image(top_image_path)
        side_img = self.preprocess_single_image(side_image_path)
        
        if top_img is None or side_img is None:
            return None
        
        # Prepare input arrays
        top_input = np.expand_dims(top_img, axis=0)
        side_input = np.expand_dims(side_img, axis=0)
        
        try:
            # Use dual-view inputs
            predictions_normalized = self.model.predict([top_input, side_input], verbose=0)
            
            # Denormalize predictions
            if self.param_scaler is not None:
                predictions_original = self.param_scaler.inverse_transform(predictions_normalized)
            else:
                predictions_original = predictions_normalized
            
            return {
                'predicted_parameters_original': predictions_original[0],
                'predicted_parameters_normalized': predictions_normalized[0],
                'parameter_names': self.param_names,
                'parameter_names_en': self.param_names_en,
                'top_image_path': top_image_path,
                'side_image_path': side_image_path
            }
            
        except Exception as e:
            print(f"Error during dual-view prediction: {e}")
            return None
    
    def preprocess_single_image(self, image_path):
        """Preprocess a single image"""
        try:
            if not os.path.exists(image_path):
                return None
            
            img = load_img(image_path, target_size=(224, 224), color_mode='rgb')
            img_array = img_to_array(img) / 255.0
            return img_array
            
        except Exception as e:
            print(f"Error preprocessing image {image_path}: {e}")
            return None
    
    def load_model_and_data(self):
        """Load the trained model and reference data"""
        print("Loading model and reference data...")
        
        try:
            # Load model with custom objects
            parameter_weights = self.param_weights
            custom_objects = {
                'weighted_mse': parameter_weighted_mse_loss(parameter_weights),
                'weighted_mae': parameter_weighted_mae_loss(parameter_weights)
            }
            
            self.model = tf.keras.models.load_model(self.model_path, custom_objects=custom_objects)
            print(f"Model loaded from {self.model_path}")
            
            # Load reference data for similarity comparison
            data_loader = PatentRegressionDataLoader(self.data_path, normalize_params=True)
            top_images, side_images, parameter_values, folder_names, image_paths = data_loader.load_regression_data()
            
            self.param_scaler = data_loader.param_scaler
            
            self.reference_data = {
                'parameters_normalized': parameter_values,
                'folder_names': folder_names,
                'image_paths': image_paths
            }
            
            if self.param_scaler is not None:
                self.reference_data['parameters_original'] = self.param_scaler.inverse_transform(parameter_values)
            else:
                self.reference_data['parameters_original'] = parameter_values
            
            print(f"Reference dataset loaded: {len(folder_names)} samples")
            
        except Exception as e:
            print(f"Error loading model or data: {e}")
            raise

In [16]:
def train_patent_regression_model():
    """Main training function for patent-based regression"""
    
    print("="*80)
    print("Patent-Based Material Parameter Regression Training")
    print("Target: Continuous parameter prediction from dual-view images")
    print("="*80)
    
    # 1. Load regression data
    print("\nStep 1: Loading patent-based regression data...")
    data_loader = PatentRegressionDataLoader('./Materials_data', normalize_params=True)
    
    top_images, side_images, parameter_values, folder_names, image_paths = data_loader.load_regression_data()
    
    if len(top_images) == 0:
        print("ERROR: No valid regression data found!")
        return None
    
    print(f"Loaded {len(top_images)} image pairs for regression training")
    print(f"Parameter shape: {parameter_values.shape}")
    
    # 2. Split data for regression
    print("\nStep 2: Creating train/validation split...")
    indices = np.arange(len(top_images))
    train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)
    
    # Split data
    top_train, top_val = top_images[train_idx], top_images[val_idx]
    side_train, side_val = side_images[train_idx], side_images[val_idx]
    param_train, param_val = parameter_values[train_idx], parameter_values[val_idx]
    
    print(f"Training set: {len(top_train)} samples")
    print(f"Validation set: {len(top_val)} samples")
    
    # 3. Create regression model
    print("\nStep 3: Creating patent-based regression model...")
    model = create_patent_regression_model(input_shape=(224, 224, 3))
    
    # Display model summary
    print("Model Architecture Summary:")
    print(f"Total Parameters: {model.count_params():,}")
    print(f"Input: Dual-view images + continuous parameter prediction")
    print(f"Output: 4 continuous parameters [弯曲强度, 强度, 形变强度, 形变率]")
    
    # 4. Compile with parameter-weighted loss
    print("\nStep 4: Compiling model with parameter-weighted loss...")
    
    # Parameter importance weights from patent
    parameter_weights = np.array([1.0, 0.6, 0.6, 0.3], dtype=np.float32)
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=parameter_weighted_mse_loss(parameter_weights),
        metrics=[
            'mae',
            parameter_weighted_mae_loss(parameter_weights)
        ]
    )
    
    # 5. Setup callbacks
    print("\nStep 5: Setting up training callbacks...")
    callbacks = create_regression_callbacks()
    
    # 6. Train model
    print("\nStep 6: Starting regression training...")
    print("Training with:")
    print("- Dual-view image inputs (top + side)")
    print("- Parameter-weighted MSE loss")
    print("- Patent-based importance weighting")
    
    history = model.fit(
        [top_train, side_train],
        param_train,
        validation_data=([top_val, side_val], param_val),
        batch_size=16,
        epochs=100,
        callbacks=callbacks,
        verbose=1
    )
    
    # 7. Evaluate model
    print("\nStep 7: Evaluating regression model...")
    results = evaluate_regression_model(
        model, 
        [top_val, side_val], 
        param_val,
        param_scaler=data_loader.param_scaler
    )
    
    # 8. Fine-tuning phase
    print("\nStep 8: Fine-tuning with unfrozen CNN layers...")
    
    # Unfreeze ResNet50 layers for fine-tuning
    for layer in model.layers:
        if 'resnet50' in layer.name:
            layer.trainable = True
            # Freeze early layers, unfreeze later ones
            for sublayer in layer.layers[:-30]:
                sublayer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss=parameter_weighted_mse_loss(parameter_weights),
        metrics=['mae', parameter_weighted_mae_loss(parameter_weights)]
    )
    
    # Fine-tune
    fine_tune_history = model.fit(
        [top_train, side_train],
        param_train,
        validation_data=([top_val, side_val], param_val),
        batch_size=8,
        epochs=30,
        callbacks=callbacks,
        verbose=1
    )
    
    # 9. Final evaluation
    print("\nStep 9: Final evaluation after fine-tuning...")
    final_results = evaluate_regression_model(
        model,
        [top_val, side_val], 
        param_val,
        param_scaler=data_loader.param_scaler
    )
    
    # 10. Save final model
    model.save('patent_regression_model_final.h5')
    print("Model saved as 'patent_regression_model_final.h5'")
    
    # 11. Plot results
    print("\nStep 10: Plotting regression results...")
    plot_regression_results(final_results)
    
    # 12. Plot training history
    plot_training_history(history, fine_tune_history)
    
    print("\n" + "="*80)
    print("Regression training completed successfully!")
    print("Files created:")
    print("- best_patent_regression_model.h5 (best model during training)")
    print("- patent_regression_model_final.h5 (final fine-tuned model)")
    print("- regression_results.png (prediction scatter plots)")
    print("- regression_training_history.png (training curves)")
    print("="*80)
    
    return model, history, fine_tune_history, final_results

In [ ]:
# Test inference with ALL images in testing folder
import os
import glob

# Initialize flexible predictor for single images
flexible_predictor = FlexiblePatentRegressionPredictor('patent_regression_model_final.h5')

# Check testing folder
testing_folder = './testing'
if not os.path.exists(testing_folder):
    print(f"Testing folder '{testing_folder}' does not exist.")
    print("Please create the folder and add your test images.")
else:
    # Find all image files in testing folder
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
    all_images = []
    
    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(testing_folder, ext)))
    
    if not all_images:
        print(f"No image files found in '{testing_folder}' folder.")
        print("Supported formats: .jpg, .jpeg, .png")
    else:
        print(f"Found {len(all_images)} test images in '{testing_folder}' folder:")
        for img in all_images:
            print(f"  {os.path.basename(img)}")
        
        print("\\n" + "="*80)
        print("TESTING ALL IMAGES")
        print("="*80)
        
        # Test each image individually
        for i, image_path in enumerate(all_images, 1):
            image_name = os.path.basename(image_path)
            
            print(f"\\n[{i}/{len(all_images)}] Testing: {image_name}")
            print("-" * 60)
            
            # Predict using single image
            prediction_result = flexible_predictor.predict_with_single_image(image_path)
            
            if prediction_result is not None:
                print(f"View type detected: {prediction_result['view_type']}")
                print(f"Note: {prediction_result['note']}")
                
                print("\\nPredicted Parameters:")
                for j, name in enumerate(flexible_predictor.param_names):
                    value = prediction_result['predicted_parameters_original'][j]
                    print(f"  {name}: {value:.2f}")
                    
            else:
                print(f"❌ Error: Could not make prediction for {image_name}")
        
        print("\\n" + "="*80)
        print("CHECKING FOR DUAL-VIEW PAIRS")
        print("="*80)
        
        # Look for dual-view pairs (top + side)
        dual_pairs = []
        image_dict = {}
        
        # Group images by base name
        for img_path in all_images:
            img_name = os.path.basename(img_path)
            
            # Try to identify pairs based on naming patterns
            if '-1.' in img_name:  # Top view
                base_name = img_name.replace('-1.', '.')
                if base_name not in image_dict:
                    image_dict[base_name] = {}
                image_dict[base_name]['top'] = img_path
            elif '-2.' in img_name:  # Side view
                base_name = img_name.replace('-2.', '.')
                if base_name not in image_dict:
                    image_dict[base_name] = {}
                image_dict[base_name]['side'] = img_path
            elif 'top' in img_name.lower():
                base_name = img_name.lower().replace('top', '').replace('_', '').replace('-', '')
                if base_name not in image_dict:
                    image_dict[base_name] = {}
                image_dict[base_name]['top'] = img_path
            elif 'side' in img_name.lower():
                base_name = img_name.lower().replace('side', '').replace('_', '').replace('-', '')
                if base_name not in image_dict:
                    image_dict[base_name] = {}
                image_dict[base_name]['side'] = img_path
        
        # Find complete pairs
        for base_name, pair_dict in image_dict.items():
            if 'top' in pair_dict and 'side' in pair_dict:
                dual_pairs.append((pair_dict['top'], pair_dict['side']))
        
        if dual_pairs:
            print(f"\\nFound {len(dual_pairs)} dual-view pairs:")
            
            # Initialize original dual-view predictor for better dual-view handling
            dual_predictor = PatentRegressionPredictor('patent_regression_model_final.h5')
            dual_predictor.load_model_and_data()
            
            for i, (top_path, side_path) in enumerate(dual_pairs, 1):
                top_name = os.path.basename(top_path)
                side_name = os.path.basename(side_path)
                
                print(f"\\n[Pair {i}] Top: {top_name} + Side: {side_name}")
                print("-" * 60)
                
                # Predict using dual-view
                dual_result = dual_predictor.predict_material_parameters(top_path, side_path)
                
                if dual_result is not None:
                    print("Dual-view Predicted Parameters:")
                    for j, name in enumerate(dual_predictor.param_names):
                        value = dual_result['predicted_parameters_original'][j]
                        print(f"  {name}: {value:.2f}")
                    
                    print("\\n✅ Dual-view prediction (higher accuracy expected)")
                else:
                    print(f"❌ Error: Could not make dual-view prediction")
        else:
            print("\\nNo dual-view pairs found.")
            print("Tip: Name your images with patterns like:")
            print("  - material1-1.jpg (top) + material1-2.jpg (side)")
            print("  - sample_top.jpg + sample_side.jpg")
        
        print("\\n" + "="*80)
        print("TESTING COMPLETED")
        print("="*80)
        print(f"Total images tested: {len(all_images)}")
        print(f"Dual-view pairs found: {len(dual_pairs)}")
        print("\\nNote: Single-view predictions may be less accurate than dual-view pairs.")

In [18]:
# Test inference with trained model using test images
predictor = FlexiblePatentRegressionPredictor('patent_regression_model_final.h5')

# Load model and reference data
predictor.load_model_and_data()

# Use test images from /testing folder
test_top_path = './testing/sample_top.jpg'  # Replace with actual filename
test_side_path = './testing/sample_side.jpg'  # Replace with actual filename

# Check what images are available
import os
has_top = os.path.exists(test_top_path)
has_side = os.path.exists(test_side_path)

if has_top and has_side:
    # Both images available - normal dual-view prediction
    print(f"Testing with dual-view images:")
    print(f"  Top view: {test_top_path}")
    print(f"  Side view: {test_side_path}")
    
    prediction_result = predictor.predict_material_parameters(test_top_path, test_side_path)
    
elif has_top and not has_side:
    # Only top view available - use top image for both views
    print(f"Only top view available: {test_top_path}")
    print(f"Using top view for both inputs (dual-view model limitation)")
    
    prediction_result = predictor.predict_material_parameters(test_top_path, test_top_path)
    
elif has_side and not has_top:
    # Only side view available - use side image for both views
    print(f"Only side view available: {test_side_path}")
    print(f"Using side view for both inputs (dual-view model limitation)")
    
    prediction_result = predictor.predict_material_parameters(test_side_path, test_side_path)
    
else:
    # No test images found
    print("No test images found. Please ensure you have at least one of:")
    print("  ./testing/sample_top.jpg")
    print("  ./testing/sample_side.jpg")
    
    # List available files in testing folder if it exists
    if os.path.exists('./testing'):
        print("\\nFiles found in ./testing folder:")
        for file in os.listdir('./testing'):
            if file.endswith(('.jpg', '.png', '.jpeg')):
                print(f"  {file}")
        print("\\nUpdate the paths above to match your actual test image filenames.")
    else:
        print("\\nNote: ./testing folder does not exist.")
    
    prediction_result = None

# Display results if prediction was successful
if prediction_result is not None:
    print("\\nPredicted Parameters:")
    for i, name in enumerate(predictor.param_names):
        print(f"  {name}: {prediction_result['predicted_parameters_original'][i]:.2f}")
    
    # Find similar materials
    similarity_results = predictor.find_similar_materials(
        prediction_result['top_image_path'], 
        prediction_result['side_image_path'], 
        top_k=3
    )
    print("\\nTop 3 Similar Materials:")
    for material in similarity_results['similar_materials']:
        print(f"  {material['rank']}. {material['material_id']} (similarity: {material['similarity_score']:.4f})")
        print(f"      Reference parameters: {material['reference_parameters']}")
        print(f"      Parameter differences: {material['parameter_differences']}")
        
    if not has_side or not has_top:
        print("\\nNote: Results may be less accurate since the model was trained on dual-view images.")
else:
    print("\\nCould not make predictions. Please check your test images.")

Loading model and reference data...
Model loaded from patent_regression_model_final.h5
Loading patent-based regression data...
Target: Continuous parameter prediction [弯曲强度, 强度, 形变强度, 形变率]
Loaded 150 image pairs for regression
Parameter shape: (150, 4)

Regression Target Statistics:
  弯曲强度 (Bending_Strength):
    Range: [0.0, 100.0]
    Mean±Std: 50.8±31.6
  强度 (Strength):
    Range: [20.0, 100.0]
    Mean±Std: 63.6±27.0
  形变强度 (Deformation_Strength):
    Range: [20.0, 60.0]
    Mean±Std: 41.3±15.9
  形变率 (Deformation_Rate):
    Range: [20.0, 100.0]
    Mean±Std: 20.5±6.5

Parameter Distribution Analysis:
  Bending_Strength: 11 unique values
  Strength: 5 unique values
  Deformation_Strength: 3 unique values
  Deformation_Rate: 2 unique values
Parameters normalized to [0,1] range
Reference dataset loaded: 150 samples
No test images found. Please ensure you have at least one of:
  ./testing/sample_top.jpg
  ./testing/sample_side.jpg
\nFiles found in ./testing folder:
  1-1.png
  2-1.png


## Example Usage

In [ ]:
# Run the complete training pipeline
model, history, fine_tune_history, results = train_patent_regression_model()

Patent-Based Material Parameter Regression Training
Target: Continuous parameter prediction from dual-view images

Step 1: Loading patent-based regression data...
Loading patent-based regression data...
Target: Continuous parameter prediction [弯曲强度, 强度, 形变强度, 形变率]
Loaded 150 image pairs for regression
Parameter shape: (150, 4)

Regression Target Statistics:
  弯曲强度 (Bending_Strength):
    Range: [0.0, 100.0]
    Mean±Std: 50.8±31.6
  强度 (Strength):
    Range: [20.0, 100.0]
    Mean±Std: 63.6±27.0
  形变强度 (Deformation_Strength):
    Range: [20.0, 60.0]
    Mean±Std: 41.3±15.9
  形变率 (Deformation_Rate):
    Range: [20.0, 100.0]
    Mean±Std: 20.5±6.5

Parameter Distribution Analysis:
  Bending_Strength: 11 unique values
  Strength: 5 unique values
  Deformation_Strength: 3 unique values
  Deformation_Rate: 2 unique values
Parameters normalized to [0,1] range
Loaded 150 image pairs for regression training
Parameter shape: (150, 4)

Step 2: Creating train/validation split...
Training set: 120

In [21]:
# Test inference with trained model
predictor = FlexiblePatentRegressionPredictor('patent_regression_model_final.h5')

# Example: Use a sample from the dataset
predictor.load_model_and_data()
sample_idx = 0
test_top_path, test_side_path = predictor.reference_data['image_paths'][sample_idx]

# Predict parameters
prediction_result = predictor.predict_material_parameters(test_top_path, test_side_path)
print("Predicted Parameters:")
for i, name in enumerate(predictor.param_names):
    print(f"  {name}: {prediction_result['predicted_parameters_original'][i]:.2f}")

# Find similar materials
similarity_results = predictor.find_similar_materials(test_top_path, test_side_path, top_k=3)
print("\nTop 3 Similar Materials:")
for material in similarity_results['similar_materials']:
    print(f"  {material['rank']}. {material['material_id']} (similarity: {material['similarity_score']:.4f})")

Loading model and reference data...
Model loaded from patent_regression_model_final.h5
Loading patent-based regression data...
Target: Continuous parameter prediction [弯曲强度, 强度, 形变强度, 形变率]
Loaded 150 image pairs for regression
Parameter shape: (150, 4)

Regression Target Statistics:
  弯曲强度 (Bending_Strength):
    Range: [0.0, 100.0]
    Mean±Std: 50.8±31.6
  强度 (Strength):
    Range: [20.0, 100.0]
    Mean±Std: 63.6±27.0
  形变强度 (Deformation_Strength):
    Range: [20.0, 60.0]
    Mean±Std: 41.3±15.9
  形变率 (Deformation_Rate):
    Range: [20.0, 100.0]
    Mean±Std: 20.5±6.5

Parameter Distribution Analysis:
  Bending_Strength: 11 unique values
  Strength: 5 unique values
  Deformation_Strength: 3 unique values
  Deformation_Rate: 2 unique values
Parameters normalized to [0,1] range
Reference dataset loaded: 150 samples


AttributeError: 'FlexiblePatentRegressionPredictor' object has no attribute 'predict_material_parameters'